In [ ]:
#| default_exp methods

# Reconciliation Methods
> Classes in this module have the `reconcile` method capable of reconcile base forecasts using `numpy` arrays.  

In [ ]:
#| export
from collections import OrderedDict
from copy import deepcopy
from typing import Dict, List

import numpy as np
from numba import njit
from statsmodels.stats.moment_helpers import cov2corr

In [ ]:
#| hide
from fastcore.test import ExceptionExpected, test_close, test_eq
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#| export
def _reconcile(S: np.ndarray, P: np.ndarray, W: np.ndarray, 
               y_hat: np.ndarray, SP: np.ndarray = None):
    if SP is None:
        SP = S @ P
    return np.matmul(SP, y_hat)

# <span style="color:DarkBlue"> 1. Bottom Up </span>

In [ ]:
#| exporti
def bottom_up(S: np.ndarray,
              y_hat: np.ndarray,
              idx_bottom: List[int]):
    n_hiers, n_bottom = S.shape
    P = np.zeros_like(S, dtype=np.float32)
    P[idx_bottom] = S[idx_bottom]
    P = P.T
    W = np.eye(n_hiers, dtype=np.float32)
    return _reconcile(S, P, W, y_hat)

In [ ]:
#| export
class BottomUp:
    """Bottom Up Reconciliation Class.
    [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

    The most basic hierarchical reconciliation is performed using an Bottom-Up strategy. It was proposed for 
    the first time by Orcutt in 1968.
    The corresponding hierarchical "projection" matrix is defined as:

    $$\mathbf{P}_{\\text{BU}} = [\mathbf{0}_{\mathrm{[b],[a]}}\;|\;\mathbf{I}_{\mathrm{[b][b]}}]$$

    
    **Parameters:**<br>
    None
    
    **References:**<br>
    - [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
    Economic Review, 58 , 773{787)](http://www.jstor.org/stable/1815532).
    """
    
    def reconcile(
            self,
            S: np.ndarray, 
            y_hat: np.ndarray, 
            idx_bottom: np.ndarray
        ):
        """Bottom Up Reconciliation Method.
        [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

        **Parameters:**<br>
        `S`: Summing matrix of size (`base`, `bottom`).<br>
        `y_hat`: Forecast values of size (`base`, `horizon`).<br>
        `idx_bottom`: Indices corresponding to the bottom level of `S`, size (`bottom`).<br>

        """
        return bottom_up(S=S, y_hat=y_hat, idx_bottom=idx_bottom)
    
    __call__ = reconcile

In [ ]:
show_doc(BottomUp, title_level=3)

<h3 id="BottomUp" class="doc_header"><code>class</code> <code>BottomUp</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>BottomUp</code>()

Bottom Up Reconciliation Class.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

The most basic hierarchical reconciliation is performed using an Bottom-Up strategy. It was proposed for 
the first time by Orcutt in 1968.
The corresponding hierarchical "projection" matrix is defined as:

$$\mathbf{P}_{\text{BU}} = [\mathbf{0}_{\mathrm{[b],[a]}}\;|\;\mathbf{I}_{\mathrm{[b][b]}}]$$


**Parameters:**<br>
None

**References:**<br>
- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
Economic Review, 58 , 773{787)](http://www.jstor.org/stable/1815532).

In [ ]:
show_doc(BottomUp.reconcile, title_level=3)

<h3 id="BottomUp.reconcile" class="doc_header"><code>BottomUp.reconcile</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h3>

> <code>BottomUp.reconcile</code>(**`S`**:`ndarray`, **`y_hat`**:`ndarray`, **`idx_bottom`**:`ndarray`)

Bottom Up Reconciliation Method.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

**Parameters:**<br>
`S`: Summing matrix of size (`base`, `bottom`).<br>
`y_hat`: Forecast values of size (`base`, `horizon`).<br>
`idx_bottom`: Indices corresponding to the bottom level of `S`, size (`bottom`).<br>

In [ ]:
#| hide
S = np.array([
    [1., 1., 1., 1.],
    [1., 1., 0., 0.],
    [0., 0., 1., 1.],
    [0., 1., 0., 0.],
    [1., 0., 0., 0.],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.],
])
h = 10
_y = np.array([10., 5., 4., 2., 1.])
y_bottom = np.vstack([i * _y for i in range(1, 5)])
y_hat_bottom_insample = np.roll(y_bottom, 1)
y_hat_bottom_insample[:, 0] = np.nan
y_hat_bottom = np.vstack([i * np.ones(h) for i in range(1, 5)])
idx_bottom = [4, 3, 5, 6]
levels = {'level1': np.array([0]),
          'level2': np.array([1, 2]),
          'level3': idx_bottom}

In [ ]:
#| hide
cls_bottom_up = BottomUp()
test_eq(
    cls_bottom_up(S=S, y_hat=S @ y_hat_bottom, idx_bottom=idx_bottom),
    S @ y_hat_bottom
)

In [ ]:
#| exporti
def is_strictly_hierarchical(S: np.ndarray, 
                             levels: Dict[str, np.ndarray]):
    # main idea:
    # if S represents a strictly hierarchical structure
    # the number of paths before the bottom level
    # should be equal to the number of nodes
    # of the previuos level
    levels_ = dict(sorted(levels.items(), key=lambda x: len(x[1])))
    # removing bottom level
    levels_.popitem()
    # making S categorical
    hiers = [np.argmax(S[idx], axis=0) + 1 for _, idx in levels_.items()]
    hiers = np.vstack(hiers)
    paths = np.unique(hiers, axis=1).shape[1] 
    nodes = levels_.popitem()[1].size
    return paths == nodes

In [ ]:
#| hide
assert is_strictly_hierarchical(S, levels)
S_non_hier = np.array([
    [1., 1., 1., 1.], # total
    [1., 1., 0., 0.], # city 1
    [0., 0., 1., 1.], # city 2
    [1., 0., 1., 0.], # transgender
    [0., 1., 0., 1.], # no transgender
    [1., 0., 0., 0.], #city 1 - transgender
    [0., 1., 0., 0.], #city 1 - no transgender
    [0., 0., 1., 0.], #city 2 - transgender
    [0., 0., 0., 1.], #city 2 - no transgender
])
levels_non_hier = {
    'Country': np.array([0]),
    'Country/City': np.array([2, 1]),
    'Country/Transgender': np.array([3, 4]),
    'Country-City-Transgender': np.array([5, 6, 7, 8]),
}
assert not is_strictly_hierarchical(S_non_hier, levels_non_hier)

In [ ]:
#| exporti
def _get_child_nodes(S: np.ndarray, levels: Dict[str, np.ndarray]):
    level_names = list(levels.keys())
    nodes = OrderedDict()
    for i_level, level in enumerate(level_names[:-1]):
        parent = levels[level]
        child = np.zeros_like(S)
        idx_child = levels[level_names[i_level+1]] 
        child[idx_child] = S[idx_child]
        nodes_level = {}
        for idx_parent_node in parent:
            parent_node = S[idx_parent_node]
            idx_node = child * parent_node.astype(bool)
            idx_node, = np.where(idx_node.sum(axis=1) > 0)
            nodes_level[idx_parent_node] = [idx for idx in idx_child if idx in idx_node]
        nodes[level] = nodes_level
    return nodes        

In [ ]:
#| exporti
def _reconcile_fcst_proportions(S: np.ndarray, y_hat: np.ndarray,
                                levels: Dict[str, np.ndarray],
                                nodes: Dict[str, Dict[int, np.ndarray]],
                                idx_top: int):
    reconciled = np.zeros_like(y_hat)
    reconciled[idx_top] = y_hat[idx_top]
    level_names = list(levels.keys())
    for i_level, level in enumerate(level_names[:-1]):
        nodes_level = nodes[level]
        for idx_parent, idx_childs in nodes_level.items():
            fcst_parent = reconciled[idx_parent]
            childs_sum = y_hat[idx_childs].sum()
            for idx_child in idx_childs:
                reconciled[idx_child] = y_hat[idx_child] * fcst_parent / childs_sum
    return reconciled

# <span style="color:DarkBlue"> 2. Top Down </span>

In [ ]:
#| exporti
def top_down(S: np.ndarray, 
             y_hat: np.ndarray,
             y_insample: np.ndarray,
             levels: Dict[str, np.ndarray],
             method: str):
    if not is_strictly_hierarchical(S, levels):
        raise ValueError('Top down reconciliation requires strictly hierarchical structures.')
    
    n_hiers, n_bottom = S.shape
    idx_top = int(S.sum(axis=1).argmax())
    levels_ = dict(sorted(levels.items(), key=lambda x: len(x[1])))
    idx_bottom = levels_[list(levels_)[-1]]
    
    if method == 'forecast_proportions':
        nodes = _get_child_nodes(S=S, levels=levels_)
        reconciled = [_reconcile_fcst_proportions(S=S, y_hat=y_hat_[:, None], 
                                                  levels=levels_, 
                                                  nodes=nodes,
                                                  idx_top=idx_top) \
                      for y_hat_ in y_hat.T]
        reconciled = np.hstack(reconciled)
        return reconciled
    else:
        y_top = y_insample[idx_top]
        y_btm = y_insample[idx_bottom]
        if method == 'average_proportions':
            prop = np.mean(y_btm / y_top, axis=1)
        elif method == 'proportion_averages':
            prop = np.mean(y_btm, axis=1) / np.mean(y_top)
        else:
            raise Exception(f'Unknown method {method}')
    P = np.zeros_like(S, np.float64).T #float 64 if prop is too small, happens with wiki2
    P[:, idx_top] = prop
    W = np.eye(n_hiers, dtype=np.float32)
    return _reconcile(S, P, W, y_hat)

In [ ]:
#| export
class TopDown:
    """Top Down Reconciliation Class.
    [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

    The Top Down hierarchical reconciliation method, distributes the total aggregate predictions and decomposes 
    it down the hierarchy using proportions $\mathbf{p}_{\mathrm{[b]}}$ that can be actual historical values 
    or estimated.
    
    $$\mathbf{P}=[\mathbf{p}_{\mathrm{[b]}}\;|\;\mathbf{0}_{\mathrm{[b][a,b\;-1]}}]$$

    **Parameters:**<br>
    `method`: One of `forecast_proportions`, `average_proportions` and `proportion_averages`.<br>
    
    **References:**<br>
    - [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
    doi:10.1002/for.3980090304](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304).<br>
    - [An investigation of aggregate variable time series forecast strategies with specific subaggregate 
    time series statistical correlation. Computers and Operations Research, 26 , 1133–1149. 
    doi:10.1016/S0305-0548(99)00017-9](https://doi.org/10.1016/S0305-0548(99)00017-9).
    """
    def __init__(
            self, 
            method: str 
        ):
        self.method = method
    
    def reconcile(
            self, 
            S: np.ndarray, 
            y_hat: np.ndarray, 
            y_insample: np.ndarray, 
            levels: Dict[str, np.ndarray] 
        ):
        """Top Down Reconciliation Method.
        [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

        **Parameters:**<br>
        `S`: Summing matrix of size (`base`, `bottom`).<br>
        `y_hat`: Forecast values of size (`base`, `horizon`).<br>
        `y_insample`: Insample values of size (`base`, `insample_size`).<br>
        `levels`: Each key is a level and each value its `S` indices.<br>

        """
        return top_down(S=S, y_hat=y_hat, 
                        y_insample=y_insample, 
                        levels=levels,
                        method=self.method)
    
    __call__ = reconcile

In [ ]:
show_doc(TopDown, title_level=3)

<h3 id="TopDown" class="doc_header"><code>class</code> <code>TopDown</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>TopDown</code>(**`method`**:`str`)

Top Down Reconciliation Class.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

The Top Down hierarchical reconciliation method, distributes the total aggregate predictions and decomposes 
it down the hierarchy using proportions $\mathbf{p}_{\mathrm{[b]}}$ that can be actual historical values 
or estimated.

$$\mathbf{P}=[\mathbf{p}_{\mathrm{[b]}}\;|\;\mathbf{0}_{\mathrm{[b][a,b\;-1]}}]$$

**Parameters:**<br>
`method`: One of `forecast_proportions`, `average_proportions` and `proportion_averages`.<br>

**References:**<br>
- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
doi:10.1002/for.3980090304](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304).<br>
- [An investigation of aggregate variable time series forecast strategies with specific subaggregate 
time series statistical correlation. Computers and Operations Research, 26 , 1133–1149. 
doi:10.1016/S0305-0548(99)00017-9](https://doi.org/10.1016/S0305-0548(99)00017-9).

In [ ]:
show_doc(TopDown.reconcile, title_level=3)

<h3 id="TopDown.reconcile" class="doc_header"><code>TopDown.reconcile</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h3>

> <code>TopDown.reconcile</code>(**`S`**:`ndarray`, **`y_hat`**:`ndarray`, **`y_insample`**:`ndarray`, **`levels`**:`Dict`\[`str`, `ndarray`\])

Top Down Reconciliation Method.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

**Parameters:**<br>
`S`: Summing matrix of size (`base`, `bottom`).<br>
`y_hat`: Forecast values of size (`base`, `horizon`).<br>
`y_insample`: Insample values of size (`base`, `insample_size`).<br>
`levels`: Each key is a level and each value its `S` indices.<br>

In [ ]:
#| hide
# we are able to recover forecasts 
# from top_down in this example
# because the time series
# share the same proportion
# across time
# but it is not a general case
for method in ['forecast_proportions', 'average_proportions', 'proportion_averages']:
    cls_top_down = TopDown(method=method)
    test_close(
        cls_top_down(
            S=S, 
            y_hat=S @ y_hat_bottom, 
            y_insample=S @ y_bottom, 
            levels=levels
        ),
        S @ y_hat_bottom
    )

# <span style="color:DarkBlue"> 3. Middle Out </span>

In [ ]:
#| exporti
def middle_out(S: np.ndarray, 
               y_hat: np.ndarray,
               y_insample: np.ndarray,
               levels: Dict[str, np.ndarray],
               level: str,
               top_down_method: str):
    if not is_strictly_hierarchical(S, levels):
        raise ValueError('Middle out reconciliation requires strictly hierarchical structures.')
    if level not in levels.keys():
        raise ValueError('You have to provide a `level` in `levels`.')
    levels_ = dict(sorted(levels.items(), key=lambda x: len(x[1])))
    reconciled = np.full_like(y_hat, fill_value=np.nan)
    cut_nodes = levels_[level]
    # bottom up reconciliation
    idxs_bu = []
    for node, idx_node in levels_.items():
        idxs_bu.append(idx_node)
        if node == level:
            break
    idxs_bu = np.hstack(idxs_bu)
    #bottom up forecasts
    bu = bottom_up(S=np.unique(S[idxs_bu], axis=1), 
                   y_hat=y_hat[idxs_bu], 
                   idx_bottom=np.arange(len(idxs_bu))[-len(cut_nodes):])
    reconciled[idxs_bu] = bu
    
    #top down
    child_nodes = _get_child_nodes(S, levels_)
    # parents contains each node in the middle out level
    # as key. The values of each node are the levels that
    # are conected to that node.
    parents = {node: {level: np.array([node])} for node in cut_nodes}
    level_names = list(levels_.keys())
    for lv, lv_child in zip(level_names[:-1], level_names[1:]):
        # if lv is not part of the middle out to bottom
        # structure we continue
        if lv not in list(parents.values())[0].keys():
            continue
        for idx_middle_out in parents.keys():
            idxs_parents = parents[idx_middle_out].values()
            complete_idxs_child = []
            for idx_parent, idxs_child in child_nodes[lv].items():
                if any(idx_parent in val for val in idxs_parents):
                    complete_idxs_child.append(idxs_child)
            parents[idx_middle_out][lv_child] = np.hstack(complete_idxs_child)
 
    for node, levels_node in parents.items():
        idxs_node = np.hstack(list(levels_node.values()))
        S_node = S[idxs_node]
        S_node = S_node[:,~np.all(S_node == 0, axis=0)]
        counter = 0
        levels_node_ = deepcopy(levels_node)
        for lv_name, idxs_level in levels_node_.items():
            idxs_len = len(idxs_level)
            levels_node_[lv_name] = np.arange(counter, idxs_len + counter)
            counter += idxs_len
        td = top_down(S_node, 
                      y_hat[idxs_node], 
                      y_insample[idxs_node], 
                      levels_node_, 
                      method=top_down_method)
        reconciled[idxs_node] = td
    return reconciled
        

In [ ]:
#| export
class MiddleOut:
    """Middle Out Reconciliation Class.
    [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

    This method is only available for strictly hierarchical structures. It anchors the base predictions in 
    a middle level. The levels above the base predictions use the bottom-up approach, while the levels below 
    use a top-down.

    **Parameters:**<br>
    `level`: Middle level.<br>
    `top_down_method`: One of `forecast_proportions`, `average_proportions` and `proportion_averages`.<br>
    
    """
    def __init__(
            self, 
            level: str, 
            top_down_method: str 
        ):
        self.level = level
        self.top_down_method = top_down_method 
    
    def reconcile(
            self, 
            S: np.ndarray, 
            y_hat: np.ndarray,
            y_insample: np.ndarray, 
            levels: Dict[str, np.ndarray] 
        ):
        """Middle Out Reconciliation Method.
        [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

        **Parameters:**<br>
        `S`: Summing matrix of size (`base`, `bottom`).<br>
        `y_hat`: Forecast values of size (`base`, `horizon`).<br>
        `y_insample`: Insample values of size (`base`, `insample_size`).<br>
        `levels`: Each key is a level and each value its `S` indices.<br>

        """
        return middle_out(S=S, y_hat=y_hat, 
                          y_insample=y_insample, 
                          levels=levels,
                          level=self.level,
                          top_down_method=self.top_down_method)
    
    __call__ = reconcile

In [ ]:
show_doc(MiddleOut, title_level=3)

<h3 id="MiddleOut" class="doc_header"><code>class</code> <code>MiddleOut</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>MiddleOut</code>(**`level`**:`str`, **`top_down_method`**:`str`)

Middle Out Reconciliation Class.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

This method is only available for strictly hierarchical structures. It anchors the base predictions in 
a middle level. The levels above the base predictions use the bottom-up approach, while the levels below 
use a top-down.

**Parameters:**<br>
`level`: Middle level.<br>
`top_down_method`: One of `forecast_proportions`, `average_proportions` and `proportion_averages`.<br>

In [ ]:
show_doc(MiddleOut.reconcile, title_level=3)

<h3 id="MiddleOut.reconcile" class="doc_header"><code>MiddleOut.reconcile</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h3>

> <code>MiddleOut.reconcile</code>(**`S`**:`ndarray`, **`y_hat`**:`ndarray`, **`y_insample`**:`ndarray`, **`levels`**:`Dict`\[`str`, `ndarray`\])

Middle Out Reconciliation Method.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

**Parameters:**<br>
`S`: Summing matrix of size (`base`, `bottom`).<br>
`y_hat`: Forecast values of size (`base`, `horizon`).<br>
`y_insample`: Insample values of size (`base`, `insample_size`).<br>
`levels`: Each key is a level and each value its `S` indices.<br>

This method is only available for strictly hierarchical structures. It anchors the base predictions in a middle level. The levels above the base predictions use the bottom-up approach, while the levels below use a top-down.

In [ ]:
#| hide
# we are able to recover forecasts 
# from middle out in this example
# because the time series
# share the same proportion
# across time
# but it is not a general case
for method in ['forecast_proportions', 'average_proportions', 'proportion_averages']:
    cls_middle_out = MiddleOut(level='level2', top_down_method=method)
    test_close(
        cls_middle_out(
            S=S, 
            y_hat=S @ y_hat_bottom, 
            y_insample=S @ y_bottom, 
            levels=levels
        ),
        S @ y_hat_bottom
    )

In [ ]:
#| exporti
def crossprod(x):
    return x.T @ x

# <span style="color:DarkBlue"> 4. MinTrace </span>

In [ ]:
#| exporti
def min_trace(S: np.ndarray, 
              y_hat: np.ndarray,
              y_insample: np.ndarray,
              y_hat_insample: np.ndarray,
              method: str):
    # shape residuals_insample (n_hiers, obs)
    res_methods = ['wls_var', 'mint_cov', 'mint_shrink']
    if method in res_methods and y_insample is None and y_hat_insample is None:
        raise ValueError(f"For methods {', '.join(res_methods)} you need to pass residuals")
    n_hiers, n_bottom = S.shape
    if method == 'ols':
        W = np.eye(n_hiers)
    elif method == 'wls_struct':
        W = np.diag(S @ np.ones((n_bottom,)))
    elif method in res_methods:
        #we need residuals with shape (obs, n_hiers)
        residuals = (y_insample - y_hat_insample).T
        n, _ = residuals.shape
        masked_res = np.ma.array(residuals, mask=np.isnan(residuals))
        covm = np.ma.cov(masked_res, rowvar=False, allow_masked=True).data
        if method == 'wls_var':
            W = np.diag(np.diag(covm))
        elif method == 'mint_cov':
            W = covm
        elif method == 'mint_shrink':
            tar = np.diag(np.diag(covm))
            corm = cov2corr(covm)
            xs = np.divide(residuals, np.sqrt(np.diag(covm)))
            xs = xs[~np.isnan(xs).any(axis=1), :]
            v = (1 / (n * (n - 1))) * (crossprod(xs ** 2) - (1 / n) * (crossprod(xs) ** 2))
            np.fill_diagonal(v, 0)
            corapn = cov2corr(tar)
            d = (corm - corapn) ** 2
            lmd = v.sum() / d.sum()
            lmd = max(min(lmd, 1), 0)
            W = lmd * tar + (1 - lmd) * covm
    else:
        raise ValueError(f'Unkown reconciliation method {method}')
    
    eigenvalues, _ = np.linalg.eig(W)
    if any(eigenvalues < 1e-8):
        raise Exception(f'min_trace ({method}) needs covariance matrix to be positive definite.')
    
    R = S.T @ np.linalg.pinv(W)
    P = np.linalg.pinv(R @ S) @ R
    
    return _reconcile(S, P, W, y_hat)

In [ ]:
#| export
class MinTrace:
    """MinTrace Reconciliation Class.
    [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

    This reconciliation algorithm proposed by Wickramasuriya et al. depends on a generalized least squares estimator 
    and an estimator of the covariance matrix of the coherency errors $\mathbf{W}_{h}$. The Min Trace algorithm 
    minimizes the squared errors for the coherent forecasts under an unbiasedness assumption; the solution has a 
    closed form.<br>
    
    $$\mathbf{P}_{\\text{MinT}}=\\left(\mathbf{S}^{\intercal}\mathbf{W}_{h}\mathbf{S}\\right)^{-1}
    \mathbf{S}^{\intercal}\mathbf{W}^{-1}_{h}$$

    **Parameters:**<br>
    `method`: One of `ols`, `wls_struct`, `wls_var`, `mint_shrink`, `mint_co`.<br>
    
    **References:**<br>
    - [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). Optimal forecast reconciliation for
    hierarchical and grouped time series through trace minimization. Journal of the American Statistical Association, 
    114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/).
    """
    def __init__(
            self, 
            method: str 
        ):
        self.method = method
        
    def reconcile(
            self, 
            S: np.ndarray, 
            y_hat: np.ndarray, 
            y_insample: np.ndarray, 
            y_hat_insample: np.ndarray 
        ):
        """MinTrace Reconciliation Method.
        [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

        **Parameters:**<br>
        `S`: Summing matrix of size (`base`, `bottom`).<br>
        `y_hat`: Forecast values of size (`base`, `horizon`).<br>
        `y_insample`: Insample values of size (`base`, `insample_size`).<br>
        `y_hat_insample`: Insample forecasts of size (`base`, `insample_size`).<br>

        """
        return min_trace(S=S, y_hat=y_hat, 
                         y_insample=y_insample,
                         y_hat_insample=y_hat_insample,
                         method=self.method)
    
    __call__ = reconcile

In [ ]:
show_doc(MinTrace, title_level=3)

<h3 id="MinTrace" class="doc_header"><code>class</code> <code>MinTrace</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>MinTrace</code>(**`method`**:`str`)

MinTrace Reconciliation Class.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

This reconciliation algorithm proposed by Wickramasuriya et al. depends on a generalized least squares estimator 
and an estimator of the covariance matrix of the coherency errors $\mathbf{W}_{h}$. The Min Trace algorithm 
minimizes the squared errors for the coherent forecasts under an unbiasedness assumption; the solution has a 
closed form.<br>

$$\mathbf{P}_{\text{MinT}}=\left(\mathbf{S}^{\intercal}\mathbf{W}_{h}\mathbf{S}\right)^{-1}
\mathbf{S}^{\intercal}\mathbf{W}^{-1}_{h}$$

**Parameters:**<br>
`method`: One of `ols`, `wls_struct`, `wls_var`, `mint_shrink`, `mint_co`.<br>

**References:**<br>
- [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). Optimal forecast reconciliation for
hierarchical and grouped time series through trace minimization. Journal of the American Statistical Association, 
114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/).

In [ ]:
show_doc(MinTrace.reconcile, title_level=3)

<h3 id="MinTrace.reconcile" class="doc_header"><code>MinTrace.reconcile</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h3>

> <code>MinTrace.reconcile</code>(**`S`**:`ndarray`, **`y_hat`**:`ndarray`, **`y_insample`**:`ndarray`, **`y_hat_insample`**:`ndarray`)

MinTrace Reconciliation Method.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

**Parameters:**<br>
`S`: Summing matrix of size (`base`, `bottom`).<br>
`y_hat`: Forecast values of size (`base`, `horizon`).<br>
`y_insample`: Insample values of size (`base`, `insample_size`).<br>
`y_hat_insample`: Insample forecasts of size (`base`, `insample_size`).<br>

In [ ]:
#| hide
for method in ['ols', 'wls_struct', 'wls_var', 'mint_shrink']:
    cls_min_trace = MinTrace(method=method)
    test_close(
        cls_min_trace(
            S=S, 
            y_hat=S @ y_hat_bottom, 
            y_insample=S @ y_bottom,
            y_hat_insample=S @ y_hat_bottom_insample
        ),
        S @ y_hat_bottom
    )
with ExceptionExpected(regex='min_trace (mint_cov)*'):
    cls_min_trace = MinTrace(method='mint_cov')
    cls_min_trace(
        S=S, 
        y_hat=S @ y_hat_bottom, 
        y_insample=S @ y_bottom,
        y_hat_insample=S @ y_hat_bottom_insample
    )

# <span style="color:DarkBlue"> 5. Optimal Combination </span>

In [ ]:
#| exporti
def optimal_combination(S: np.ndarray, 
                        y_hat: np.ndarray,
                        method: str,
                        y_insample: np.ndarray = None,
                        y_hat_insample: np.ndarray = None):
    
    return min_trace(S=S, y_hat=y_hat, 
                         y_insample=y_insample,
                         y_hat_insample=y_hat_insample,
                         method=method)

In [ ]:
#| export
class OptimalCombination:
    """Optimal Combination Reconciliation Class.
    [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

    This reconciliation algorithm was proposed by Hyndman et al. 2011, the method uses generalized least squares 
    estimator using the coherency errors covariance matrix. Consider the covariance of the base forecast 
    $\\textrm{Var}(\epsilon_{h}) = \Sigma_{h}$, the $\mathbf{P}$ matrix of this method is defined by:

    $$ \mathbf{P} = \\left(\mathbf{S}^{\intercal}\Sigma_{h}^{\dagger}\mathbf{S}\\right)^{-1}\mathbf{S}^{\intercal}\Sigma^{\dagger}_{h}$$

    where $\Sigma_{h}^{\dagger}$ denotes the variance pseudo-inverse. The method was later proven equivalent to 
    `MinTrace` variants.

    **Parameters:**<br>
    `method`: Allowed Optimal Combination Methods: 'ols', 'wls_struct'.<br>
    
    **References:**<br>
    - [Rob J. Hyndman, Roman A. Ahmed, George Athanasopoulos, Han Lin Shang. "Optimal Combination Forecasts for 
    Hierarchical Time Series" (2010).](https://robjhyndman.com/papers/Hierarchical6.pdf).<br>
    - [Shanika L. Wickramasuriya, George Athanasopoulos and Rob J. Hyndman. "Optimal Combination Forecasts for 
    Hierarchical Time Series" (2010).](https://robjhyndman.com/papers/MinT.pdf).
    """
    def __init__(
            self, 
            method: str
        ):
        comb_methods = ['ols', 'wls_struct']
        if method not in comb_methods:
            raise ValueError(f"Optimal Combination class does not support method: \"{method}\"")
        
        self.method = method
    
    def reconcile(
            self,
            S: np.ndarray, 
            y_hat: np.ndarray, 
            y_insample: np.ndarray = None, 
            y_hat_insample: np.ndarray = None 
        ):
        """Optimal Combination Reconciliation Method.
        [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

        **Parameters:**<br>
        `S`: Summing matrix of size (`base`, `bottom`).<br>
        `y_hat`: Forecast values of size (`base`, `horizon`).<br>
        `y_insample`: Insample values of size (`base`, `insample_size`).<br>
        `y_hat_insample`: Insample forecasts of size (`base`, `insample_size`).<br>

        """
        return optimal_combination(S=S, 
                                   y_hat=y_hat, 
                                   y_insample=y_insample, 
                                   y_hat_insample=y_hat_insample, 
                                   method=self.method)
    
    __call__ = reconcile

In [ ]:
show_doc(OptimalCombination, title_level=3)

<h3 id="OptimalCombination" class="doc_header"><code>class</code> <code>OptimalCombination</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>OptimalCombination</code>(**`method`**:`str`)

Optimal Combination Reconciliation Class.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

This reconciliation algorithm was proposed by Hyndman et al. 2011, the method uses generalized least squares 
estimator using the coherency errors covariance matrix. Consider the covariance of the base forecast 
$   extrm{Var}(\epsilon_{h}) = \Sigma_{h}$, the $\mathbf{P}$ matrix of this method is defined by:

$$ \mathbf{P} = \left(\mathbf{S}^{\intercal}\Sigma_{h}^{\dagger}\mathbf{S}ight)^{-1}\mathbf{S}^{\intercal}\Sigma^{\dagger}_{h}$$

where $\Sigma_{h}^{\dagger}$ denotes the variance pseudo-inverse. The method was later proven equivalent to 
[`MinTrace`](/hierarchicalforecast/methods.html#MinTrace) variants.

**Parameters:**<br>
`method`: Allowed Optimal Combination Methods: 'ols', 'wls_struct'.<br>

**References:**<br>
- [Rob J. Hyndman, Roman A. Ahmed, George Athanasopoulos, Han Lin Shang. "Optimal Combination Forecasts for 
Hierarchical Time Series" (2010).](https://robjhyndman.com/papers/Hierarchical6.pdf).<br>
- [Shanika L. Wickramasuriya, George Athanasopoulos and Rob J. Hyndman. "Optimal Combination Forecasts for 
Hierarchical Time Series" (2010).](https://robjhyndman.com/papers/MinT.pdf).

In [ ]:
show_doc(OptimalCombination.reconcile, title_level=3)

<h3 id="OptimalCombination.reconcile" class="doc_header"><code>OptimalCombination.reconcile</code><a href="__main__.py#L34" class="source_link" style="float:right">[source]</a></h3>

> <code>OptimalCombination.reconcile</code>(**`S`**:`ndarray`, **`y_hat`**:`ndarray`, **`y_insample`**:`ndarray`=*`None`*, **`y_hat_insample`**:`ndarray`=*`None`*)

Optimal Combination Reconciliation Method.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

**Parameters:**<br>
`S`: Summing matrix of size (`base`, `bottom`).<br>
`y_hat`: Forecast values of size (`base`, `horizon`).<br>
`y_insample`: Insample values of size (`base`, `insample_size`).<br>
`y_hat_insample`: Insample forecasts of size (`base`, `insample_size`).<br>

In [ ]:
#| hide
for method in ['ols', 'wls_struct']:
    cls_optimal_combination = OptimalCombination(method=method)
    test_close(
        cls_optimal_combination(
            S=S, 
            y_hat=S @ y_hat_bottom 
        ),
        S @ y_hat_bottom
    )

In [ ]:
#| exporti
@njit
def lasso(X: np.ndarray, y: np.ndarray, 
          lambda_reg: float, max_iters: int = 1_000,
          tol: float = 1e-4):
    # lasso cyclic coordinate descent
    n, feats = X.shape
    norms = (X ** 2).sum(axis=0)
    beta = np.zeros(feats, dtype=np.float32)
    beta_changes = np.zeros(feats, dtype=np.float32)
    residuals = y.copy()
    
    for it in range(max_iters):
        for i, betai in enumerate(beta):
            # is feature is close to zero, we 
            # continue to the next.
            # in this case is optimal betai= 0
            if abs(norms[i]) < 1e-8:
                continue
            xi = X[:, i]
            #we calculate the normalized derivative
            rho = betai + xi.flatten().dot(residuals) / norms[i] #(norms[i] + 1e-3)
            #soft threshold
            beta[i] = np.sign(rho) * max(np.abs(rho) - lambda_reg * n / norms[i], 0.)#(norms[i] + 1e-3), 0.)
            beta_changes[i] = np.abs(betai - beta[i])
            if beta[i] != betai:
                residuals += (betai - beta[i]) * xi
        if max(beta_changes) < tol:
            break
    #print(it)
    return beta

# <span style="color:DarkBlue"> 6. ERM </span>

In [ ]:
#| exporti
def erm(S: np.ndarray,
        y_hat: np.ndarray,
        y_insample: np.ndarray,
        y_hat_insample: np.ndarray,
        idx_bottom: np.ndarray,
        method: str,
        lambda_reg: float = 1e-3):
    n_hiers, n_bottom = S.shape
    # y_hat_insample shape (n_hiers, obs)
    # remove obs with nan values
    nan_idx = np.isnan(y_hat_insample).any(axis=0)
    y_insample = y_insample[:, ~nan_idx]
    y_hat_insample = y_hat_insample[:, ~nan_idx]
    #only using h validation steps to avoid 
    #computational burden
    #print(y_hat.shape)
    h = min(y_hat.shape[1], y_hat_insample.shape[1])
    y_hat_insample = y_hat_insample[:, -h:] # shape (h, n_hiers)
    y_insample = y_insample[:, -h:]
    if method == 'closed':
        B = np.linalg.inv(S.T @ S) @ S.T @ y_insample
        B = B.T
        P = np.linalg.pinv(y_hat_insample.T) @ B
        P = P.T
    elif method in ['reg', 'reg_bu']:
        X = np.kron(np.array(S, order='F'), np.array(y_hat_insample.T, order='F'))
        Pbu = np.zeros_like(S)
        if method == 'reg_bu':
            Pbu[idx_bottom] = S[idx_bottom]
        Pbu = Pbu.T
        Y = y_insample.T.flatten(order='F') - X @ Pbu.T.flatten(order='F')
        if lambda_reg is None:
            lambda_reg = np.max(np.abs(X.T.dot(Y)))
        P = lasso(X, Y, lambda_reg)
        P = P + Pbu.T.flatten(order='F')
        P = P.reshape(-1, n_bottom, order='F').T
    else:
        raise ValueError(f'Unkown reconciliation method {method}')
        
    W = np.eye(n_hiers, dtype=np.float32)
    
    return _reconcile(S, P, W, y_hat)

In [ ]:
#| export
class ERM:
    """Optimal Combination Reconciliation Class.
    [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

    The Empirical Risk Minimization reconciliation strategy relaxes the unbiasedness assumptions from
    previous reconciliation methods like MinT and optimizes square errors between the reconciled predictions
    and the validation data to obtain an optimal reconciliation matrix P.

    The exact solution for $\mathbf{P}$ (`method='closed'`) follows the expression:

    $$\mathbf{P}^{*} = \\left(\mathbf{S}^{\intercal}\mathbf{S}\\right)^{-1}\mathbf{Y}^{\intercal}\hat{\mathbf{Y}}\\left(\hat{\mathbf{Y}}\hat{\mathbf{Y}}\\right)^{-1}$$

    The alternative Lasso regularized $\mathbf{P}$ solution (`method='reg_bu'`) is useful when the observations 
    of validation data is limited or the exact solution has low numerical stability.

    $$\mathbf{P}^{*} = \\text{argmin}_{\mathbf{P}} ||\mathbf{Y}-\mathbf{S} \mathbf{P} \hat{Y} ||^{2}_{2} + \lambda ||\mathbf{P}-\mathbf{P}_{\\text{BU}}||_{1}$$

    **Parameters:**<br>
    `method`: one of `closed`, `reg` and `reg_bu`.<br>
    `lambda_reg`: l1 regularizer for `reg` and `reg_bu`.<br>
    
    **References:**<br>
    - [Ben Taieb, S., & Koo, B. (2019). Regularized regression for hierarchical forecasting without 
    unbiasedness conditions. In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge 
    Discovery & Data Mining KDD '19 (p. 1337{1347). New York, NY, USA: Association for Computing Machinery.]
    (https://doi.org/10.1145/3292500.3330976).<br>
    """
    def __init__(
            self, 
            method: str, 
            lambda_reg: float = 1e-2 
        ):
        self.method = method
        self.lambda_reg = lambda_reg
        
    def reconcile(
            self, 
            S: np.ndarray, 
            y_hat: np.ndarray, 
            y_insample: np.ndarray, 
            y_hat_insample: np.ndarray, 
            idx_bottom: np.ndarray 
        ):
        """ERM Reconciliation Method.
        [Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

        **Parameters:**<br>
        `S`: Summing matrix of size (`base`, `bottom`).<br>
        `y_hat`: Forecast values of size (`base`, `horizon`).<br>
        `y_insample`: Insample values of size (`base`, `insample_size`).<br>
        `y_hat_insample`: Insample forecasts of size (`base`, `insample_size`).<br>
        `idx_bottom`: Indices corresponding to the bottom level of `S`, size (`bottom`).<br>
        """
        return erm(S=S, y_hat=y_hat, 
                   y_insample=y_insample,
                   y_hat_insample=y_hat_insample,
                   idx_bottom=idx_bottom,
                   method=self.method, lambda_reg=self.lambda_reg)
    
    __call__ = reconcile

In [ ]:
show_doc(ERM, title_level=3)

<h3 id="ERM" class="doc_header"><code>class</code> <code>ERM</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>ERM</code>(**`method`**:`str`, **`lambda_reg`**:`float`=*`0.01`*)

Optimal Combination Reconciliation Class.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

The Empirical Risk Minimization reconciliation strategy relaxes the unbiasedness assumptions from
previous reconciliation methods like MinT and optimizes square errors between the reconciled predictions
and the validation data to obtain an optimal reconciliation matrix P.

The exact solution for $\mathbf{P}$ (`method='closed'`) follows the expression:

$$\mathbf{P}^{*} = \left(\mathbf{S}^{\intercal}\mathbf{S}ight)^{-1}\mathbf{Y}^{\intercal}\hat{\mathbf{Y}}\left(\hat{\mathbf{Y}}\hat{\mathbf{Y}}ight)^{-1}$$

The alternative Lasso regularized $\mathbf{P}$ solution (`method='reg_bu'`) is useful when the observations 
of validation data is limited or the exact solution has low numerical stability.

$$\mathbf{P}^{*} =  ext{argmin}_{\mathbf{P}} ||\mathbf{Y}-\mathbf{S} \mathbf{P} \hat{Y} ||^{2}_{2} + \lambda ||\mathbf{P}-\mathbf{P}_{      ext{BU}}||_{1}$$

**Parameters:**<br>
`method`: one of `closed`, `reg` and `reg_bu`.<br>
`lambda_reg`: l1 regularizer for `reg` and `reg_bu`.<br>

**References:**<br>
- [Ben Taieb, S., & Koo, B. (2019). Regularized regression for hierarchical forecasting without 
unbiasedness conditions. In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge 
Discovery & Data Mining KDD '19 (p. 1337{1347). New York, NY, USA: Association for Computing Machinery.]
(https://doi.org/10.1145/3292500.3330976).<br>

In [ ]:
show_doc(ERM.reconcile, title_level=3)

<h3 id="ERM.reconcile" class="doc_header"><code>ERM.reconcile</code><a href="__main__.py#L37" class="source_link" style="float:right">[source]</a></h3>

> <code>ERM.reconcile</code>(**`S`**:`ndarray`, **`y_hat`**:`ndarray`, **`y_insample`**:`ndarray`, **`y_hat_insample`**:`ndarray`, **`idx_bottom`**:`ndarray`)

ERM Reconciliation Method.
[Source code](https://github.com/dluuo/hierarchicalforecast/blob/main/hierarchicalforecast/methods.py).

**Parameters:**<br>
`S`: Summing matrix of size (`base`, `bottom`).<br>
`y_hat`: Forecast values of size (`base`, `horizon`).<br>
`y_insample`: Insample values of size (`base`, `insample_size`).<br>
`y_hat_insample`: Insample forecasts of size (`base`, `insample_size`).<br>
`idx_bottom`: Indices corresponding to the bottom level of `S`, size (`bottom`).<br>

In [ ]:
#| hide
for method in ['reg_bu']:
    cls_erm = ERM(method=method, lambda_reg=None)
    test_close(
        cls_erm(
            S=S, 
            y_hat=S @ y_hat_bottom,
            y_insample=S @ y_bottom,
            y_hat_insample=S @ y_hat_bottom_insample,
            idx_bottom=idx_bottom
        ),
        S @ y_hat_bottom
    )

# <span style="color:DarkBlue"> References </span>

#### Bottom Up
-  [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition". OTexts: Melbourne, Australia. OTexts.com/fpp3  Accessed on July 2022](https://otexts.com/fpp3/).

- [Shmueli, G., & Lichtendahl Jr, K. C. (2016). "Practical time series forecasting with R: A hands-on guide". Axelrod Schnall Publishers](https://www.forecastingbook.com/).

#### Top Down
- [Rob J. Hyndman, Yeasmin Khandakar (2008). "Automatic Time Series Forecasting: The forecast package for R"](https://www.jstatsoft.org/article/view/v027i03).
